# Capstone Project - The Battle of Neighborhoods - Week 2

## Index

1. Introduction
2. Data Understanding
3. Data Preparation & Cleaning
4. Evaluation
5. Deployment
6. Conclusions

## 1. Introduction

New York City has the biggest population in the United States of America. Around 8,622,357 inhabitants. The population density is 27,755.25 people per square mile, with an area of 468.19 square miles. Food culture include an array of international cuisines influenced by the city’s immigrant history. In this study we will analyze public data to find insight, to open a new restaurant.

### Project Goals

- What is the best location in New York City for a Mexican restaurant?
- Which areas have potential Restaurant Market?
- Which all areas lack Mexican Restaurants?

## 2. Data Understanding  

New York City's have a diverse offer when it comes to restaurants, each belonging to different categories/countries or regions. For this project, we will use an the following open dataset: 

New York City data that contains a list of Boroughs and Neighborhoods along with their gps latitude and longitude.

Dataset: https://cocl.us/new_york_dataset

Restaurants in each neighborhood of New York City.

Data set: Foursquare API

GeoSpace data set

Data set : https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm
Description: Get the New York Borough boundaries.

## 3. Data Preparation and Cleaning

### 3.1 Install and import necesary libraries

In [151]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium
# ! pip install folium==2.11.2
import folium # plotting library

# associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


### 3.2 Foursquare function

Define a function to interact with FourSquare API and get top 100 venues within a radius of 5000 metres for a given latitude and longitude. The function will return venue id, venue name and category:

In [152]:
CLIENT_ID = 'UMAHH11LV5ZCHZOSPLPTXCRMMH25ARQKISU2DGVAWRM3QWYN'
CLIENT_SECRET = '4QLBJT0HWROGTCJUEB144WIVJJ5DY31CFVLRPKBUU0LKN4BL'
VERSION = '20180604'
LIMIT = 100
radius=5000

In [153]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude

In [154]:
# Place where we get the max ratings
address = 'New York, Manhattan'
geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

Geograpical coordinate of New York, Manhattan are 40.7896239, -73.9598939.


In [159]:
def get_venues(lat,lng):
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

Test Function:

In [164]:
venues = get_venues(40.7896239, -73.9598939)
venues.head()

,ID,Name,Category
0,4ba233dbf964a5206fe337e3,East Meadow,Field
1,4a746fb2f964a52025de1fe3,The Jewish Museum,Museum
2,5328a418498e4be839240b62,Da Capo,Café
3,4a229fa8f964a520797d1fe3,Jacqueline Kennedy Onassis Reservoir,Reservoir
4,4a2fc4d3f964a520da981fe3,Cooper Hewitt Smithsonian Design Museum,Art Museum


In [148]:
venues.shape

(100, 3)

Define a function to get venue details like like count , rating , tip counts for a given venue id. This will be used for ranking:

In [165]:
def get_venue_details(venue_id):
  
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

Test Function:

In [167]:
get_venue_details(venue_id='4a2fc4d3f964a520da981fe3')

KeyError: 'venue'

In [ ]:
def get_new_york_data():
  
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data

Call funtion to get the New York City data:

In [ ]:
new_york_data = get_new_york_data()
new_york_data.head()

In [ ]:
new_york_data.shape

Saving New York data in csv format:

In [ ]:
new_york_data.to_csv('new_york_data.csv', index=False)

Plotting the Neighborhoods in New York.

In [ ]:
plt.figure(figsize=(10,5), dpi=100)
# title
plt.title('Number of Neighborhoods for each Borough in New York City')
#On x-axis
plt.xlabel('Borough', fontsize=15)
#On y-axis
plt.ylabel('No. of Neighborhoods', fontsize=15)
#giving a bar plot
new_york_data.groupby('Borough')['Neighborhood'].count().plot(kind='bar', color='green')
#legend
plt.legend()
#displays the plot
plt.show()

According to the plot, Queens has the higher number of neighborhoods while Manhattan has the lower.

Lets find out how many Mexican Restaurants are in each neighborhood:

In [ ]:
# prepare neighborhood list that contains mexican resturants
column_names = ['Borough', 'Neighborhood', 'ID','Name']
mexican_rest_ny = pd.DataFrame(columns=column_names)
count = 1
for row in new_york_data.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    mexican_resturants = venues[venues['Category']=='Mexican Restaurant']   
    print('(',count,'/',len(new_york_data),')','Mexican Resturants in '+Neighborhood+', '+Borough+':'+str(len(mexican_resturants)))
    for resturant_detail in mexican_resturants.values.tolist():
        id, name , category = resturant_detail
        mexican_rest_ny = mexican_rest_ny.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1

Lets explore Mexican Restaurants:

In [ ]:
mexican_rest_ny.head()

In [ ]:
mexican_rest_ny.shape

Saving the dataset into a csv file:

In [ ]:
mexican_rest_ny.to_csv('mexican_rest_ny.csv', index=False)

## Exploring Mexican Restaurants in each borough and neighborhood

The following plot shows the number of Mexican resturants for each Borough in New York City:

In [ ]:
plt.figure(figsize=(10,5), dpi=100)
# title
plt.title('Number of Mexican Resturants for each Borough in New York City')
#On x-axis
plt.xlabel('Borough', fontsize=15)
#On y-axis
plt.ylabel('No.of Mexican Resturants', fontsize=15)
#giving a bar plot
mexican_rest_ny.groupby('Borough')['ID'].count().plot(kind='bar', color='orangered')
#legend
plt.legend()
#displays the plot
plt.show()

According to the plot, the Bronx has the higher number of Mexican Restaurants while Manhattan the lower.

The following plot shows the number of Mexican resturants for each Neighborhood in New York City:

In [ ]:
plt.figure(figsize=(10,5), dpi=100)
# title
plt.title('Number of Mexican Resturants for each Neighborhood in New York City')
#On x-axis
plt.xlabel('Neighborhood', fontsize=15)
#On y-axis
plt.ylabel('No.of Mexican Resturants', fontsize=15)
#giving a bar plot
brazilian_rest_ny.groupby('Neighborhood')['ID'].count().nlargest(5).plot(kind='bar', color='limegreen')
#legend
plt.legend()
#displays the plot
plt.show()

As we can see in the plot, Concourse and Soundview have the highes (and equal) number of Mexican Restaurants while Borough Park, Clason Point and Glendale have the lowes (and equal) number.

Lests furhter explore Concourse and Soundview restaurants:

In [ ]:
mexican_rest_ny[mexican_rest_ny['Neighborhood']=='Concourse']

In [ ]:
mexican_rest_ny[mexican_rest_ny['Neighborhood']=='Soundview']

Lets get the ranking for each restaurant:

In [ ]:
# create a neighborhood list that contains mexican resturants
column_names=['Borough', 'Neighborhood', 'ID','Name','Likes','Rating','Tips']
mexican_rest_stats_ny=pd.DataFrame(columns=column_names)

count = 1

for row in mexican_rest_ny.values.tolist():
    Borough,Neighborhood,ID,Name=row
    try:
        venue_details=get_venue_details(ID)
        print(venue_details)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
        # we will assign 0 value for these resturants as they may have been recently opened or details does not exist in FourSquare Database
        id,name,likes,rating,tips=[0]*5
    print('(',count,'/',len(mexican_rest_ny),')','processed')
    mexican_rest_stats_ny = mexican_rest_stats_ny.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)
    count+=1

Lets see this data in a dataframe:

In [ ]:
mexican_rest_stats_ny.head()

Show dataframe shape:

In [ ]:
mexican_rest_stats_ny.shape

Now the dataframe is saved into a csv file for further exploration:

In [ ]:
mexican_rest_stats_ny.to_csv('mexican_rest_stats_ny.csv', index=False)

mexican_rest_stats_ny_csv=pd.read_csv('mexican_rest_stats_ny.csv')

In [ ]:
mexican_rest_stats_ny_csv.head()

Lets explore the datatypes of the dataframe:

In [ ]:
mexican_rest_stats_ny_csv.info()

Data type of 'Likes' and 'Tips' are of type 'int64'. We will convert them into type 'Float' for further analysis:

In [ ]:
mexican_rest_stats_ny['Likes']=mexican_rest_stats_ny_csv['Likes'].astype('float64')
mexican_rest_stats_ny['Tips']=mexican_rest_stats_ny_csv['Tips'].astype('float64')

Lets verify the changes:

In [ ]:
mexican_rest_stats_ny_csv.info()

Lets find the restuarants with Highest number of Likes, Ratings and Tips:

In [ ]:
# Resturant with highest number of Likes
mexican_rest_stats_ny.iloc[mexican_rest_stats_ny['Likes'].idxmax()]

In [ ]:
# Resturant with highest number of Rating
mexican_rest_stats_ny.iloc[mexican_rest_stats_ny['Rating'].idxmax()]

In [ ]:
# Resturant with highest number of Tips
mexican_rest_stats_ny.iloc[mexican_rest_stats_ny['Tips'].idxmax()]

Lets visualize the neighborhood with the highest average ratings for Mexican resturants:

In [ ]:
ny_borough_stats=mexican_rest_stats_ny.groupby('Borough',as_index=False).mean()[['Borough','Rating']]
ny_borough_stats.columns=['Borough','Average Rating']
ny_borough_stats.sort_values(['Average Rating'],ascending=False).head()

Lets create a plot to visualize this data:

In [ ]:
plt.figure(figsize=(10,5), dpi=100)
# title
plt.title('Average rating of Mexican Resturants for each Borough')
#On x-axis
plt.xlabel('Borough', fontsize=15)
#On y-axis
plt.ylabel('Average Rating', fontsize=15)
#giving a bar plot
mexican_rest_stats_ny.groupby('Borough').mean()['Rating'].plot(kind='bar', color='aqua')
#legend
plt.legend()
#displays the plot
plt.show()

Lets find neighborhoods with average rating greater or equal to 7.0 to visualize on a map:

In [ ]:
ny_neighborhood_stats = ny_neighborhood_stats[ny_neighborhood_stats['Average Rating']>=7.0]
ny_neighborhood_stats

Now lets join this dataset to new york data to get lonitude and latitude:

In [ ]:
ny_neighborhood_stats = pd.merge(ny_neighborhood_stats,new_york_data, on='Neighborhood')
ny_neighborhood_stats = ny_neighborhood_stats[['Borough','Neighborhood','Latitude','Longitude','Average Rating']]
ny_neighborhood_stats

Lets map the data:

In [ ]:
ny_map = folium.Map(location=geo_location('New York'), zoom_start=10)

In [ ]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the neighborhood and add each to the feature group
for lat, lng, in ny_neighborhood_stats[['Latitude','Longitude']].values:
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10, # define how big the circle markers will be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

Add a new field to dataframe for labeling purpose:

In [ ]:
ny_neighborhood_stats['Label']=ny_neighborhood_stats['Neighborhood']+', '+ny_neighborhood_stats['Borough']+'('+ny_neighborhood_stats['Average Rating'].map(str)+')'

Add pop-up text to each marker on the map:

In [ ]:
for lat, lng, label in ny_neighborhood_stats[['Latitude','Longitude','Label']].values:
    folium.Marker([lat, lng], popup=label).add_to(ny_map)        
# add incidents to map
ny_map.add_child(incidents)

Make this Notebook Trusted to load map: File -> Trust Notebook

Now lets visualize boroughs based on average rating:

In [ ]:
ny_map2 = folium.Map(location=geo_location('New York'), zoom_start=10)

In [ ]:
# instantiate a feature group for the incidents in the dataframe
incidents2 = folium.map.FeatureGroup()

# loop through the neighborhood and add each to the feature group
for lat, lng, in ny_neighborhood_stats[['Latitude','Longitude']].values:
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10, # define how big the circle markers will be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

In [ ]:
ny_neighborhood_stats['Label']=ny_neighborhood_stats['Neighborhood']+', '+ny_neighborhood_stats['Borough']+'('+ny_neighborhood_stats['Average Rating'].map(str)+')'


In [ ]:
for lat, lng, label in ny_neighborhood_stats[['Latitude','Longitude','Label']].values:
    folium.Marker([lat, lng], popup=label).add_to(ny_map)        
# add incidents to map
ny_map.add_child(incidents)

## 4. Evaluation

After all the analysis that was perform we have found the needed information to decide where would be the ideal place to open the new restaurant.

## 5. Deployment

This project could be implemented into a regression algorithm for further development.

## 6. Conclusions

This project has led us to conclude that:

- Manhattan is the best place to open the restaurant since there is little competition
- Along with Manhattan, Queens and Brooklyn have good market potential for a new restaurant
- There aren't areas lacking Mexican Restaurants but there are areas with low number of them